# Cryptocurrencies Analysis

### Importing libraries:

In [1]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.pyplot as plt

### Importing data from Wikipedia - List of cryptocurrencies

In [2]:
#Extracting the information from the web page.
res = requests.get("https://en.wikipedia.org/wiki/List_of_cryptocurrencies")
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
datos=soup.find_all('table')[0]
datos.find_all('tr')[1].find_all("td")[0]

<td>2009
</td>

In [4]:
def createDF(table):
    
    headers = [header.text[:-1] for header in datos.find_all('th')]
    data=[]
    for row in table.find_all('tr')[1:]:
        currency = {}
        for index,value in enumerate(row.find_all("td")):
            currency[headers[index]]=value.text[:-1]
        data.append(currency)
    data = pd.DataFrame(data)
    return data

wiki=createDF(datos)

In [5]:
def quitarBrackets():
    for columna in wiki:
        wiki[columna] = wiki[columna].str.replace(r"\[.*\]","")
    return wiki
clean_wiki = quitarBrackets()

In [21]:
def changeWiki():    
    for i in range(len(clean_wiki)):

        diccionario = {'EOS.IO':'EOS',
                       'Bitcoin-cash':'Bitcoin Cash', 
                       'Ether or "Ethereum"':'Ethereum'
                      }
        for x,y in diccionario.items(): 
                clean_wiki['Currency'].values[i] = clean_wiki['Currency'].values[i].replace(x, y)
    return clean_wiki

clean_wiki=changeWiki()

### Importing data from Kaggle - Historical data

In [22]:
historical_data = pd.read_csv("INPUT/consolidated_coin_data.csv",encoding='latin1')

In [23]:
def changeKaggle():    
    for i in range(len(historical_data)):

        diccionario = {'bitcoin':'Bitcoin',
                       'bitcoin-cash':'Bitcoin Cash', 
                       'eos':'EOS',
                       'ethereum':'Ethereum',
                       'litecoin':'Litecoin',
                       'stellar':'Stellar',
                       'tether':'Tether',
                       'xrp':'Ripple'
                      }
        for x,y in diccionario.items(): 
                historical_data['Currency'].values[i] = historical_data['Currency'].values[i].replace(x, y)
    return historical_data

clean_kaggle=changeKaggle()
clean_kaggle

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,tezos,"Dec 04, 2019",1.29,1.32,1.25,1.25,"46,048,752","824,588,509"
1,tezos,"Dec 03, 2019",1.24,1.32,1.21,1.29,"41,462,224","853,213,342"
2,tezos,"Dec 02, 2019",1.25,1.26,1.20,1.24,"27,574,097","817,872,179"
3,tezos,"Dec 01, 2019",1.33,1.34,1.25,1.25,"24,127,567","828,296,390"
4,tezos,"Nov 30, 2019",1.31,1.37,1.31,1.33,"28,706,667","879,181,680"
...,...,...,...,...,...,...,...,...
28939,Bitcoin-sv,"May 02, 2013",3.78,4.04,3.01,3.37,0,"58,287,979"
28940,Bitcoin-sv,"May 01, 2013",4.29,4.36,3.52,3.80,0,"65,604,596"
28941,Bitcoin-sv,"Apr 30, 2013",4.40,4.57,4.17,4.30,0,"74,020,918"
28942,Bitcoin-sv,"Apr 29, 2013",4.37,4.57,4.23,4.38,0,"75,388,964"


# CONCLUSIONS - Merging both files

In [32]:
def merging():
    result=pd.merge(clean_wiki,
                   clean_kaggle,
                   on='Currency')
    return result[['Release','Currency','Symbol','Programming language of implementation','Notes','Date','Close','Market Cap']]
final_table=merging()

In [38]:
final_table.to_csv("OUTPUT/final_table.csv")